In [1]:
pip install pandas numpy ta yfinance matplotlib openpyxl



  Using cached yfinance-0.2.61-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached peewee-3.18.1.tar.gz (3.0 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached curl_cffi-0.11.1-cp39-abi3-win_amd64.whl.metadata (15 kB)
Using cached yfinance-0.2.61-py2.py3-none-any.whl (117 kB)
Using cached curl_cffi-0.11.1-cp39-abi3-win_amd64.whl (1.4 MB)
Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
  Created wheel for peewee: filename=peewee-3.18.1-py3-none-any.whl size=139100 sha256=ac61931fc9f47237b0debca322e09287c7d9762b1c179d7dc700873021b1c3aa
  Stored in directory: c:\users\anand\appdata\local\pip\cache\wheels\1a\57\6a\

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ta.momentum import RSIIndicator

# Load your CSV (1-minute data)
df = pd.read_csv("intraday_data.csv", parse_dates=["datetime"])
df.set_index("datetime", inplace=True)

# Resample to 10-min candles
df_10min = df.resample("10T").agg({
    "open": "first",
    "high": "max",
    "low": "min",
    "close": "last",
    "volume": "sum",
    "oi_change": "mean"
}).dropna()

# Add 7-period Moving Average
df_10min["MA7"] = df_10min["close"].rolling(7).mean()

# Add RSI (optional)
df_10min["RSI"] = RSIIndicator(df_10min["close"]).rsi()

# Apply filters
# Step 1: OI > 7%
df_10min = df_10min[df_10min["oi_change"] > 7]

# Step 2: Price change filter (from open to 10:00 AM)
initial_price = df_10min.iloc[0]["open"]
pct_change = (df_10min.iloc[1]["close"] - initial_price) / initial_price * 100

if abs(pct_change) < 2:
    print("Price change < 2%. No trade.")
else:
    # Check candle relation with MA7
    if df_10min.iloc[0]["close"] > df_10min.iloc[0]["MA7"] and df_10min.iloc[1]["close"] > df_10min.iloc[1]["MA7"]:
        signal = "BUY"
        entry_price = df_10min.iloc[1]["close"]
        sl = df_10min.iloc[1]["low"]  # stop loss below second candle
        tp = entry_price + 2 * (entry_price - sl)  # target = 1:2
    elif df_10min.iloc[0]["close"] < df_10min.iloc[0]["MA7"] and df_10min.iloc[1]["close"] < df_10min.iloc[1]["MA7"]:
        signal = "SELL"
        entry_price = df_10min.iloc[1]["close"]
        sl = df_10min.iloc[1]["high"]  # stop loss above second candle
        tp = entry_price - 2 * (sl - entry_price)  # target = 1:2
    else:
        signal = "NO TRADE"
        entry_price = None

    print(f"Signal: {signal}")
    if signal != "NO TRADE":
        print(f"Entry: {entry_price:.2f}, SL: {sl:.2f}, TP: {tp:.2f}")


FileNotFoundError: [Errno 2] No such file or directory: 'intraday_data.csv'

In [3]:
from upstox_api.api import Upstox, Session
import pandas as pd
from ta.trend import SMAIndicator
import time

# ---- USER SETUP ----
API_KEY = "YOUR_API_KEY"
API_SECRET = "YOUR_API_SECRET"
REDIRECT_URI = "https://your-redirect.com"
ACCESS_TOKEN = "YOUR_ACCESS_TOKEN"  # After login, use the returned token here

# ---- Step 1: Login ----
u = Upstox(API_KEY, API_SECRET)
u.set_access_token(ACCESS_TOKEN)

# ---- Step 2: Define Instruments ----
nifty_50 = [ # Add more symbols or fetch via API
    'RELIANCE', 'TCS', 'ICICIBANK', 'INFY', 'HDFCBANK'
]

# ---- Step 3: Strategy Logic ----
def get_signal(stock):
    try:
        historical = u.get_ohlc(u.get_instrument_by_symbol('NSE_EQ', stock), interval='5minute', count=20)
        df = pd.DataFrame(historical)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df.set_index('timestamp', inplace=True)
        df = df[['open', 'high', 'low', 'close', 'volume']]

        # Add SMA(7)
        df['MA7'] = SMAIndicator(df['close'], window=7).sma_indicator()

        # OI & % change should be fetched separately (if supported)
        # For now, assume OI change is simulated or fetched externally

        candle1 = df.iloc[-3]
        candle2 = df.iloc[-2]

        if candle1['close'] > candle1['MA7'] and candle2['close'] > candle2['MA7']:
            return "BUY", candle2['close']
        elif candle1['close'] < candle1['MA7'] and candle2['close'] < candle2['MA7']:
            return "SELL", candle2['close']
        else:
            return "HOLD", None
    except Exception as e:
        print(f"Error for {stock}: {e}")
        return "HOLD", None

# ---- Step 4: Auto Trade ----
def execute_trades():
    for stock in nifty_50:
        signal, price = get_signal(stock)
        if signal == "BUY":
            instrument = u.get_instrument_by_symbol('NSE_EQ', stock)
            u.place_order(TransactionType.Buy,
                          instrument,
                          quantity=1,
                          order_type=OrderType.Market,
                          product=ProductType.Intraday,
                          duration=DurationType.DAY)
            print(f"Bought {stock} at market price")
        elif signal == "SELL":
            instrument = u.get_instrument_by_symbol('NSE_EQ', stock)
            u.place_order(TransactionType.Sell,
                          instrument,
                          quantity=1,
                          order_type=OrderType.Market,
                          product=ProductType.Intraday,
                          duration=DurationType.DAY)
            print(f"Sold {stock} at market price")
        else:
            print(f"No trade for {stock}")

# ---- Step 5: Run at 10:15 AM ----
import schedule
schedule.every().day.at("10:15").do(execute_trades)

print("📈 AI Trading Bot Running...")
while True:
    schedule.run_pending()
    time.sleep(60)


ModuleNotFoundError: No module named 'upstox_api'

In [5]:
# === XAUUSD PROFIT-ONLY TRADING BOT (MT5 + Python) ===

import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import talib
import time

# === CONFIG ===
SYMBOL = "XAUUSD"
TIMEFRAME = mt5.TIMEFRAME_M15
LOT_SIZE = 0.1
MIN_PROFIT_USD = 1  # close trade when profit >= $1

# === INIT ===
if not mt5.initialize():
    raise RuntimeError(f"MT5 initialize() failed: {mt5.last_error()}")

account = mt5.account_info()
if account is None:
    raise RuntimeError(f"MT5 account_info() failed: {mt5.last_error()}")

symbol_info = mt5.symbol_info(SYMBOL)
if symbol_info is None:
    raise RuntimeError(f"Symbol {SYMBOL} not found")
if not symbol_info.visible:
    mt5.symbol_select(SYMBOL, True)

# === HELPER: Get recent candles ===
def get_data(symbol, timeframe, n=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, n)
    if rates is None or len(rates) == 0:
        raise RuntimeError(f"Failed to retrieve market data: {mt5.last_error()}")
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# === HELPER: Entry Signal (Engulfing + RSI < 40 Buy or RSI > 60 Sell) ===
def get_trade_signal(df):
    close = df['close'].astype(float).values
    open_ = df['open'].astype(float).values
    high = df['high'].astype(float).values
    low = df['low'].astype(float).values

    rsi = talib.RSI(close, timeperiod=14)
    engulf = talib.CDLENGULFING(open_, high, low, close)

    if len(rsi) == 0 or len(engulf) == 0:
        return None

    print(f"Last RSI: {rsi[-1]:.2f}, Last Engulfing: {engulf[-1]}")

    if engulf[-1] > 0 and rsi[-1] < 40:
        print("Buy signal detected")
        return "buy"
    elif engulf[-1] < 0 and rsi[-1] > 60:
        print("Sell signal detected")
        return "sell"
    else:
        print("No valid signal at this time.")
    return None

# === HELPER: Open Trade ===
def open_trade(direction):
    tick = mt5.symbol_info_tick(SYMBOL)
    price = tick.ask if direction == "buy" else tick.bid
    order_type = mt5.ORDER_TYPE_BUY if direction == "buy" else mt5.ORDER_TYPE_SELL

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": LOT_SIZE,
        "type": order_type,
        "price": price,
        "deviation": 20,
        "magic": 10001,
        "comment": "profit_only_trade",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    if result.retcode == mt5.TRADE_RETCODE_DONE:
        print(f"Opened {direction.upper()} at {price}")
    else:
        print(f"Order failed: {result.retcode} - {result.comment if hasattr(result, 'comment') else 'No comment'}")
        print(result._asdict())

# === HELPER: Close Trade if profit >= $1 ===
def check_and_close():
    positions = mt5.positions_get(symbol=SYMBOL)
    if positions:
        pos = positions[0]
        profit = pos.profit
        print(f"Current open position profit: ${profit:.2f}")
        if profit >= MIN_PROFIT_USD:
            close_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
            price = mt5.symbol_info_tick(SYMBOL).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(SYMBOL).ask
            close_req = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": SYMBOL,
                "volume": pos.volume,
                "type": close_type,
                "position": pos.ticket,
                "price": price,
                "deviation": 20,
                "magic": 10001,
                "comment": "close_in_profit",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }
            result = mt5.order_send(close_req)
            if result.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"Trade closed in profit: ${profit:.2f}")
            else:
                print(f"Close failed: {result.retcode}")

# === MAIN LOOP ===
print("Running XAUUSD profit-only bot...")
try:
    while True:
        check_and_close()
        time.sleep(2)

        if not mt5.positions_total():
            df = get_data(SYMBOL, TIMEFRAME, 100)
            signal = get_trade_signal(df)
            if signal:
                open_trade(signal)

        time.sleep(60)  # wait 1 min before checking again
except KeyboardInterrupt:
    print("Stopped by user")
finally:
    mt5.shutdown()


RuntimeError: MT5 initialize() failed: (-10005, 'IPC timeout')